# Worksheet 05

Name:  Prathmesh Sonawane

UID: U39215370

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

Firstly, we assign the values 2 and 0 as our centroids for our 2 clusters. Then, we assign every value in our dataset to its closest cluster, that being 2 and 0. In this case, the values 0 and 0.5 will be assigned to the cluster with centroid 0, and the values 1.5, 2, 6, 6.5, and 7 will be assigned to the cluster with centroid 2. Next, the mean value of all the points in each cluster will be calculated. For the cluster with centroid 0, our average will be 0.25, and for our other cluster, it will be 4.6. These two values will become the new centroids for our two clusters, and the whole process will start again. This will continue to happen until our two centroids reach a convergence point, with little to no change. 

In the second iteration, the points 0, 0.5, 1.5, and 2 will be assigned to the cluster with centroid 0.25. In addition, the points 6, 6.5, and 7 will be assigned to the cluster with centroid 4.6. Taking the mean of the points in each cluster, the new centroid becomes 1 and 6.5 respectively. 

In the third iteration, the points 0, 0.5, 1.5, and 2 will be assigned to the cluster with centroid 1. In addition, the points 6, 6.5, and 7 will be assigned to the cluster with centroid 6.5. Taking the mean of the points in each cluster, the new centroid continues to be 1 and 6.5 respectively. Since our centroids have reached a convergence, we are done and have our two clusters. 
 

b) Describe in plain english what the cost function for k means is.

The cost-function is how we evaluate and compare solutions to a given problem. In our context, for k-means clustering, we want a cost function that gives us a smaller cost when our solution is better, or more meaningful. A common cost function for partitional clustering is minimizing the total distance between all points in a node for all given nodes. 

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

For the same number of clusters k, you could have different solutions due to the randomness of picking your initial centroids for your k clusters. Due to this randomness, when we reach a convergence, we do so by finding the optimal centroids from where we started. However, if we can start with different initial centroids, we can end with different ending centroids as well. 

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, this algorithm always does converge. In every iteration of the algorithm, we find new centroids based on the mean of the previous data points in our clusters during the last iteration. This means that during every iteration, we get closer to finding the optimal centroid for every cluster. With this guarantee, we always step toward the solution, and thus we will eventually find it. 

e) Follow along in class the implementation of Kmeans

In [1]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
        
    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            
kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

[[ 2.52223629 -3.31696117]
 [ 1.79254085  2.08068856]
 [-2.75431214  1.67400929]
 [ 0.45122083 -1.90484772]]
[[ 2.08517021 -4.22909513]
 [ 1.66067199  1.82996945]
 [-2.8062043   1.6770794 ]
 [ 0.26463481 -1.09043737]]
[[ 1.87857639 -4.2014945 ]
 [ 1.72390637  1.88773635]
 [-2.90101537  1.7680821 ]
 [ 0.1661884  -0.49673396]]
[[ 1.84012274 -4.13675116]
 [ 1.79856883  1.95373897]
 [-2.98474421  1.81545992]
 [ 0.06354201 -0.2029132 ]]
[[ 1.84012274 -4.13675116]
 [ 1.88073218  1.9820243 ]
 [-3.04862622  1.85340542]
 [ 0.00773286 -0.06380801]]
[[ 1.84012274e+00 -4.13675116e+00]
 [ 1.91248297e+00  2.00204407e+00]
 [-3.07703402e+00  1.85266958e+00]
 [ 1.33733963e-02 -3.71451966e-04]]
[[ 1.84012274e+00 -4.13675116e+00]
 [ 1.91248297e+00  2.00204407e+00]
 [-3.07703402e+00  1.85266958e+00]
 [ 1.33733963e-02 -3.71451966e-04]]
